# WS 08 17-03-25

> Name: Rishi Vora
>
> Roll No: MS21113

In [2]:
import re
import math
import numpy as np

## RegEx

### Write regular expression to match:

#### a. Date given as `dd-mm-yyyy` or `dd/mm/yyyy` or `dd.mm.yyyy` format.

In [4]:
date = "22/11/2003"
re.search(r"\d{2}(\/|\-|\.)\d{2}\1\d{4}", date)

<re.Match object; span=(0, 10), match='22/11/2003'>

#### b. Email ID (name@domain.rest) [**name** can be alphanumeric, **domain** and **rest** are alphabets]

In [3]:
email = "abc123ef@somedomain.sometld"
re.search(r"\w+@[a-zA-Z]+\.[a-zA-Z]+", email)

<re.Match object; span=(0, 27), match='abc123ef@somedomain.sometld'>

#### c. Extract this information from the text below: Mr. X date of birth is **02/10/1989** and email id is **test@gmail.com**.

In [6]:
text = "Mr. X date of birth is 02/10/1989 and email id is test@gmail.com."
m=re.search(r"(\d{2}(\/|\-|\.)\d{2}\2\d{4}).*?(\w+@[a-zA-Z]+\.[a-zA-Z]+)", text)
m.groups()

('02/10/1989', '/', 'test@gmail.com')

### Rewrite to Homo Sapiens to H. sapiens or any similar names of organisms (general regex)

In [7]:
organism = "Homo sapiens"
re.sub(r"(\w)\w*\s(\w+)", r"\1. \2", organism)

'H. sapiens'

### Rewrite '>CAA57801.1= GFP [Mus Musculus]' to '>CAA57801_M.Musculus'.
Note: there could be any number of characters/space between >ID. and '[' symbol

In [8]:
somestr = ">CAA57801.1= GFP [Mus Musculus]"
re.sub(r"(>\w+)\..*\[(\w)\w*\s(\w+)\]", r"\1_\2.\3", somestr)

'>CAA57801_M.Musculus'

### Rewrite ">NM_001300425.1 Drosophila melanogaster Akt kinase (Akt), transcript variant E, mRNA" as ">NM_001300425_D.melanogaster_Akt_mRNA". Note: the >NM is fixed for any mRNA, the transcript variant may or may not be present i.e. the character after a variant is optional.
- The general format is '>NM_DIGITS.ONE-TWO_DIGITS TAXON SPECIES LONG-NAME (SHORT_NAME), transcript variant OPTIONAL_TYPE, mRNA
- LONG-NAME-multiple words, SHORT_NAME-one word OPTIONAL_TYPE-one single word

In [9]:
someotherstr = ">NM_001300425.1 Drosophila melanogaster Akt kinase (Akt), transcript variant, mRNA"
re.sub(r"(>NM\w+)\.\d{1,2} (\w)\w* (\w+) .* \((\w+)\), transcript variant( \w)?, mRNA", r"\1_\2.\3_\4_mRNA", someotherstr)

'>NM_001300425_D.melanogaster_Akt_mRNA'

## Reading PDB file

### Write a program to read the content of the PDB file (**atom_cord.pdb.txt**) into the _dict_ data type in the following form:

```
x[RESID][ATOMNAME]=[Xcord,Ycord,Zcord]
x[RESID]['resname']=RESNAME
```

for instance, using `x[5]['resname']` should return residue name and `x[5][CA]` should return list with coordinates.

```
RESID: 23-26
RESNAME:18-20
ATOMNAME:12-16
Xcord: 31-38
Ycord: 39-46
Zcord: 47-54
```

In [10]:
def parse_pdb_file(file_path):
    pdb_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                atom_name = line[12:16].strip()
                res_name = line[17:20].strip()
                res_id = int(line[22:26].strip())

                x_coord = float(line[30:38].strip())
                y_coord = float(line[38:46].strip())
                z_coord = float(line[46:54].strip())

                if res_id not in pdb_dict:
                    pdb_dict[res_id] = {'resname': res_name}

                pdb_dict[res_id][atom_name] = [x_coord, y_coord, z_coord]

    return pdb_dict

In [ ]:
pdb_dict = parse_pdb_file("atom_cord.pdb.txt")

In [12]:
pdb_dict

{5: {'resname': 'LEU',
  'N': [52.074, -24.356, 94.664],
  'CA': [52.675, -23.892, 93.371],
  'C': [51.757, -24.145, 92.178],
  'O': [51.735, -23.358, 91.233],
  'CB': [54.06, -24.527, 93.144]},
 6: {'resname': 'SER',
  'N': [50.975, -25.219, 92.233],
  'CA': [50.019, -25.489, 91.174],
  'C': [49.116, -24.273, 91.082],
  'O': [48.592, -23.981, 90.023],
  'CB': [49.184, -26.734, 91.47],
  'OG': [48.14, -26.895, 90.523]},
 7: {'resname': 'ALA',
  'N': [48.953, -23.576, 92.207],
  'CA': [48.171, -22.339, 92.272],
  'C': [48.806, -21.216, 91.454],
  'O': [48.129, -20.576, 90.657],
  'CB': [48.025, -21.898, 93.71]},
 8: {'resname': 'GLU',
  'N': [50.096, -20.979, 91.71],
  'CA': [50.862, -19.946, 91.003],
  'C': [50.913, -20.271, 89.513],
  'O': [50.71, -19.39, 88.664],
  'CB': [52.295, -19.847, 91.564],
  'CG': [53.171, -18.738, 90.936],
  'CD': [54.611, -18.738, 91.471],
  'OE1': [54.799, -18.669, 92.71],
  'OE2': [55.563, -18.815, 90.655]},
 9: {'resname': 'ASP',
  'N': [51.145, -21.55, 

### After reading the content, the program should be able to do the following:

#### Find the composition of the residues. (occurrence of amino acids in the PDB file).

In [13]:
def find_residue_composition(pdb_dict):
    composition = {}

    residue_names = [pdb_dict[res_id]['resname'] for res_id in pdb_dict]
    unique_residue_names = set(residue_names)

    for resname in unique_residue_names:
        composition[resname] = residue_names.count(resname)

    return composition

In [14]:
find_residue_composition(pdb_dict)

{'ARG': 18,
 'VAL': 19,
 'MET': 8,
 'TYR': 13,
 'GLY': 17,
 'CYS': 9,
 'PRO': 4,
 'ALA': 32,
 'GLN': 13,
 'LEU': 27,
 'ASP': 26,
 'THR': 20,
 'GLU': 28,
 'SER': 17,
 'TRP': 3,
 'HIS': 6,
 'LYS': 30,
 'PHE': 19,
 'ILE': 26,
 'ASN': 15}

#### Find whether a residue has the same number of atoms. Find the average number of atoms.

In [15]:
def analyze_atom_count(pdb_dict):
    atom_counts = {}

    for res_id, residue in pdb_dict.items():
        atom_counts[res_id] = len(residue) - 1  # Subtract 1 for 'resname' key

    # Check if all residues have the same number of atoms
    same_atoms = len(set(atom_counts.values())) == 1

    avg_atoms = sum(atom_counts.values()) / len(atom_counts)

    return same_atoms, avg_atoms

In [16]:
same_atoms, avg_atoms = analyze_atom_count(pdb_dict)
print(f"Same number of atoms in all residues: {same_atoms}")
print(f"Average number of atoms per residue: {avg_atoms}")

Same number of atoms in all residues: False
Average number of atoms per residue: 8.002857142857144


#### Find the average distance between two consecutive CA residues using the formula:

v1=[x1,x2,x3] and v2=[y1,y2,y3]

vdiff=v2-v1 (vector difference)

d=sqrt(vdiff.vdiff)

In [17]:
def calculate_distance(v1, v2):
    vdiff = [v2[i] - v1[i] for i in range(3)]
    squared_sum = sum(diff**2 for diff in vdiff)
    return math.sqrt(squared_sum)

In [18]:
def avg_CA_distance(pdb_dict):
    CA_distances = np.array([])

    for resid in list(pdb_dict.keys())[:-1] :
        v1 = pdb_dict[resid]["CA"]
        v2 = pdb_dict[resid+1]["CA"]
        distance = calculate_distance(v1, v2)
        CA_distances = np.append(CA_distances, distance)

    avg_CA_dist = np.average(CA_distances)

    return CA_distances, avg_CA_dist

In [19]:
CA_distances, avg_CA_dist = avg_CA_distance(pdb_dict)
print(avg_CA_dist)

3.800196374847075


## Sequence Alignment

### Perform sequence alignment of 'ATGCC' and 'AGC' using Dynamic Programming algorithm. Use Match score of 3, Mismatch score 0, and gap score of -1.

In [20]:
def initialize(rows, cols, scores):
    grid = np.zeros((rows+1, cols+1), dtype=int)
    gap_score = scores["gap"]

    grid[0] = [i*gap_score for i in range(cols+1)]
    grid[:,0] = [i*gap_score for i in range(rows+1)]

    return grid

In [21]:
def fill(grid, top, left, scores):
    directions = np.full(grid.shape, "", dtype="U3")

    for (row,col),elem in np.ndenumerate(grid):
        if row == 0:
            directions[row, col] = "L"
            continue
        elif col == 0:
            directions[row, col] = "U"
            continue

        S_ij = {
            "D": grid[row-1, col-1] + (scores["match"] if top[col-1] == left[row-1] else scores["mismatch"]),
            "L": grid[row, col-1] + scores["gap"],
            "U": grid[row-1, col] + scores["gap"]
        }
        m = max(S_ij.values())
        grid[row, col] = m

        direction = ""
        for key, value in S_ij.items():
            if value == m:
                direction += key
        directions[row, col] = direction

    return grid, directions

In [ ]:
def traverse(directions, paths) :
    new_paths = []

    for path in paths :
        i, j = path[-1]

        if i == 0 and j == 0 :
            return paths

        dir_ij = directions[i, j]

        if 'D' in dir_ij :
            new_paths.append(path[:-1] + [(i,j,'D')] + [(i-1,j-1)])
        if 'U' in dir_ij :
            new_paths.append(path[:-1] + [(i,j,'U')] + [(i-1,j)])
        if 'L' in dir_ij :
            new_paths.append(path[:-1] + [(i,j,'L')] + [(i,j-1)])

    return traverse(directions, new_paths)

In [23]:
def candidates(paths, top, left):
    candidates = []

    for path in paths:
        aligned = {
            "top": "",
            "left": ""
        }
        for direction in path[:-1]:
            row, col, direction = direction
            if direction == "D":
                aligned["top"] += top[col-1]
                aligned["left"] += left[row-1]
            elif direction == "L":
                aligned["top"] += top[col-1]
                aligned["left"] += "_"
            elif direction == "U":
                aligned["top"] += "_"
                aligned["left"] += left[row-1]

        aligned["top"] = aligned["top"][::-1]
        aligned["left"] = aligned["left"][::-1]
        candidates.append(aligned)

    return candidates

In [24]:
top = "GCATGCG"
left = "GATTACA"

scores = {
    "match": 3,
    "mismatch": 0,
    "gap": -1
}

In [25]:
grid_init = initialize(len(left), len(top), scores)
grid_init

array([[ 0, -1, -2, -3, -4, -5, -6, -7],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [-2,  0,  0,  0,  0,  0,  0,  0],
       [-3,  0,  0,  0,  0,  0,  0,  0],
       [-4,  0,  0,  0,  0,  0,  0,  0],
       [-5,  0,  0,  0,  0,  0,  0,  0],
       [-6,  0,  0,  0,  0,  0,  0,  0],
       [-7,  0,  0,  0,  0,  0,  0,  0]])

In [26]:
grid_fill, directions = fill(grid_init, top, left, scores)
directions

array([['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'],
       ['U', 'D', 'L', 'L', 'L', 'DL', 'L', 'DL'],
       ['U', 'U', 'D', 'D', 'L', 'L', 'L', 'L'],
       ['U', 'U', 'DU', 'U', 'D', 'L', 'L', 'L'],
       ['U', 'U', 'DU', 'U', 'DU', 'D', 'DL', 'DL'],
       ['U', 'U', 'DU', 'D', 'U', 'DU', 'D', 'DL'],
       ['U', 'U', 'D', 'U', 'U', 'DU', 'D', 'L'],
       ['U', 'U', 'U', 'D', 'LU', 'DU', 'U', 'D']], dtype='<U3')

In [27]:
traversal_start = (len(left),len(top))
obtained_paths = traverse(directions, [[traversal_start]])
for path in obtained_paths :
    print(path)

[(7, 7, 'D'), (6, 6, 'D'), (5, 5, 'D'), (4, 4, 'D'), (3, 3, 'U'), (2, 3, 'D'), (1, 2, 'L'), (1, 1, 'D'), (0, 0)]
[(7, 7, 'D'), (6, 6, 'D'), (5, 5, 'D'), (4, 4, 'U'), (3, 4, 'D'), (2, 3, 'D'), (1, 2, 'L'), (1, 1, 'D'), (0, 0)]
[(7, 7, 'D'), (6, 6, 'D'), (5, 5, 'U'), (4, 5, 'D'), (3, 4, 'D'), (2, 3, 'D'), (1, 2, 'L'), (1, 1, 'D'), (0, 0)]


In [28]:
candidates(obtained_paths, top, left)

[{'top': 'GCA_TGCG', 'left': 'G_ATTACA'},
 {'top': 'GCAT_GCG', 'left': 'G_ATTACA'},
 {'top': 'GCATG_CG', 'left': 'G_ATTACA'}]